# Лабораторная работа №6
## Задание:
1 . Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
### Подключение библиотек


In [1]:
!pip install catboost

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from catboost import CatBoostClassifier
import matplotlib.pyplot as plt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.8 MB/s eta 0:00:00


### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('cybersecurity_attacks.csv')  # Замени на путь к файлу
print(Dataset.head())

             Timestamp Source IP Address Destination IP Address  Source Port  \
0  2023-05-30 06:33:58     103.216.15.12           84.9.164.252        31225   
1  2020-08-26 07:08:30    78.199.217.198         66.191.137.154        17245   
2  2022-11-13 08:23:25      63.79.210.48          198.219.82.17        16811   
3  2023-07-02 10:38:46     163.42.196.10        101.228.192.255        20018   
4  2023-07-16 13:11:07     71.166.185.76        189.243.174.238         6131   

   Destination Port Protocol  Packet Length Packet Type Traffic Type  \
0             17616     ICMP            503        Data         HTTP   
1             48166     ICMP           1174        Data         HTTP   
2             53600      UDP            306     Control         HTTP   
3             32534      UDP            385        Data         HTTP   
4             26646      TCP           1462        Data          DNS   

                                        Payload Data  ... Action Taken  \
0  Qui natus

### Создание искомого столбца

In [3]:
label_encoder = LabelEncoder()
Dataset['Attack Type Encoded'] = label_encoder.fit_transform(Dataset['Attack Type'])
print(Dataset[['Attack Type', 'Attack Type Encoded']].head(10))

  Attack Type  Attack Type Encoded
0     Malware                    2
1     Malware                    2
2        DDoS                    0
3     Malware                    2
4        DDoS                    0
5     Malware                    2
6        DDoS                    0
7   Intrusion                    1
8   Intrusion                    1
9     Malware                    2


### Нормализация данных

In [4]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)

# Заполнение пустых значений средними значениями
columns_to_fill = ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Source Port          40000 non-null  int64  
 1   Destination Port     40000 non-null  int64  
 2   Packet Length        40000 non-null  int64  
 3   Anomaly Scores       40000 non-null  float64
 4   Attack Type Encoded  40000 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 1.5 MB


### Поиск лучшей метрики и количества соседей

In [5]:
X = Dataset.drop(columns=['Attack Type Encoded'])
y = Dataset['Attack Type Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Параметры для настройки CatBoost
catboost_params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [4, 6, 8]
}

# Создание модели CatBoost
catboost_model = CatBoostClassifier(verbose=False)

# Подбор лучших параметров с помощью GridSearchCV
catboost_grid_search = GridSearchCV(catboost_model, catboost_params, cv=5)
catboost_grid_search.fit(X_train, y_train)

# Лучшие параметры
best_catboost_params = catboost_grid_search.best_params_
print(f"Лучшие параметры для CatBoost: {best_catboost_params}")


Лучшие параметры для CatBoost: {'depth': 4, 'iterations': 200, 'learning_rate': 0.05}


#### Обучение на лучших метриках и количестве соседей

In [6]:
# Переобучение модели с лучшими параметрами на всем тренировочном наборе данных
best_catboost_model = CatBoostClassifier(**best_catboost_params, verbose=False)
best_catboost_model.fit(X_train, y_train)

# Предсказание на тестовых данных
y_pred_catboost = best_catboost_model.predict(X_test)

# Оценка точности
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
print(f"Точность CatBoost на тестовых данных: {accuracy_catboost}")

# Отчет о классификации для CatBoost
print("Classification Report для CatBoost:")
print(classification_report(y_test, y_pred_catboost))

Точность CatBoost на тестовых данных: 0.32875
Classification Report для CatBoost:
              precision    recall  f1-score   support

           0       0.33      0.40      0.36      5306
           1       0.34      0.22      0.26      5416
           2       0.33      0.37      0.35      5278

    accuracy                           0.33     16000
   macro avg       0.33      0.33      0.32     16000
weighted avg       0.33      0.33      0.32     16000

